In [5]:
import pandas as pd
import numpy as np

In [6]:
trainfile ='mnist_train.csv'
testfile ='mnist_test.csv'

In [7]:
df_train = pd.read_csv(trainfile,header=None)
df_test = pd.read_csv(testfile,header=None)

In [8]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
def transform(x):
    if x<255/2:
        return 0
    else:
        return 1

In [10]:
df_train.iloc[:,1:] = df_train.iloc[:,1:].applymap(transform)
df_test.iloc[:,1:] = df_test.iloc[:,1:].applymap(transform)

In [11]:
train_y = df_train.iloc[:,0]
train_x = df_train.iloc[:,1:]

In [12]:
P_class = train_y.value_counts(sort=False)/len(train_y)
P_class

5    0.090350
0    0.098717
4    0.097367
1    0.112367
9    0.099150
2    0.099300
3    0.102183
6    0.098633
7    0.104417
8    0.097517
Name: 0, dtype: float64

In [13]:
train_y==0

0        False
1         True
2        False
3        False
4        False
         ...  
59995    False
59996    False
59997    False
59998    False
59999    False
Name: 0, Length: 60000, dtype: bool

In [14]:
train_x[train_y==0]

,1,2,3,4,5,6,7,8,9,10,...,775,776,777,778,779,780,781,782,783,784
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59940,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59944,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59952,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59972,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
f_sum = train_x[train_y==0].apply(np.sum,axis=0)
f_sum

1      0
2      0
3      0
4      0
5      0
      ..
780    0
781    0
782    0
783    0
784    0
Length: 784, dtype: int64

In [17]:
c_num = train_y.value_counts(sort=False)
c_num

5    5421
0    5923
4    5842
1    6742
9    5949
2    5958
3    6131
6    5918
7    6265
8    5851
Name: 0, dtype: int64

In [18]:
c_0 = c_num[0]
c_0

5923

In [19]:
k,N = 1,2
P_fi_c0 = (f_sum +k)/(c_0+k*N)
P_fi_c0[:5]

1    0.000169
2    0.000169
3    0.000169
4    0.000169
5    0.000169
dtype: float64

In [20]:
def train(X, Y, k=1,N=2,C=10):
    
    #  参数说明：
    #  X：样本特征，pandas.DataFrame类型
    #  Y: 样本标签，pandas.Series类型
    #  k,N : 拉普拉斯平滑参数
    #  C: 类别数
    
    #用numpy.array计算，
    #都返回array类型
    '''
    P_class = np.array(Y.value_counts(sort=False)) #shape:(C,)
    I_fc    = np.array([X[Y==i].apply(np.sum,axis=0).tolist() for i in range(10)]) + k #shape:(10,784)
    I_c     = P_class + N*k
    E       = I_fc/I_c[:,None] 
    
    #用pandas中DataFrame,Series计算，
    #返回：P_class（Series），E（DataFrame）
    '''
    P_class = Y.value_counts(sort=False)
    I_fc    = pd.DataFrame([X[Y==i].apply(np.sum,axis=0).tolist() for i in range(C)]) + k
    I_c     = P_class + N*k
    E       = I_fc.div(I_c,axis=0).values  #结果为 np.array 类型
    
    
    
    return P_class,E

In [21]:
P_class,E = train(train_x,train_y)

In [22]:
P_class

5    5421
0    5923
4    5842
1    6742
9    5949
2    5958
3    6131
6    5918
7    6265
8    5851
Name: 0, dtype: int64

In [23]:
pd.DataFrame(E)

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0.000169,0.000169,0.000169,0.000169,0.000169,0.000169,0.000169,0.000169,0.000169,0.000169,...,0.000169,0.000169,0.000169,0.000169,0.000169,0.000169,0.000169,0.000169,0.000169,0.000169
1,0.000148,0.000148,0.000148,0.000148,0.000148,0.000148,0.000148,0.000148,0.000148,0.000148,...,0.000148,0.000148,0.000148,0.000148,0.000148,0.000148,0.000148,0.000148,0.000148,0.000148
2,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,...,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168
3,0.000163,0.000163,0.000163,0.000163,0.000163,0.000163,0.000163,0.000163,0.000163,0.000163,...,0.000163,0.000163,0.000163,0.000163,0.000163,0.000163,0.000163,0.000163,0.000163,0.000163
4,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,...,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171
5,0.000184,0.000184,0.000184,0.000184,0.000184,0.000184,0.000184,0.000184,0.000184,0.000184,...,0.000184,0.000184,0.000184,0.000184,0.000184,0.000184,0.000184,0.000184,0.000184,0.000184
6,0.000169,0.000169,0.000169,0.000169,0.000169,0.000169,0.000169,0.000169,0.000169,0.000169,...,0.000169,0.000169,0.000169,0.000169,0.000169,0.000169,0.000169,0.000169,0.000169,0.000169
7,0.000160,0.000160,0.000160,0.000160,0.000160,0.000160,0.000160,0.000160,0.000160,0.000160,...,0.006223,0.002234,0.001277,0.000479,0.000479,0.000160,0.000160,0.000160,0.000160,0.000160
8,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,...,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171
9,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,0.000168,...,0.000840,0.000840,0.001008,0.000504,0.000336,0.000168,0.000168,0.000168,0.000168,0.000168


In [37]:
sample_x = df_test.iloc[0,1:].values.reshape(1,-1) #取一个测试样本
sample_y = df_test.iloc[0,0]                       #样本标签
P_class,E = train(train_x,train_y)

In [38]:
sample_x

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [28]:
bz1 = np.subtract(sample_x,1)
bz2 = np.abs(bz1)
bz3 = np.subtract(bz2,E)
bz4 = np.abs(bz3)
bz4.shape

(10, 784)

In [30]:
P_class

5    5421
0    5923
4    5842
1    6742
9    5949
2    5958
3    6131
6    5918
7    6265
8    5851
Name: 0, dtype: int64

In [39]:
log_p_fi_c0 = np.log(bz4)       #取对数
log_p_c0    = np.log(P_class)
P_c0_X      = log_p_c0 + np.sum(log_p_fi_c0,axis=1)
P_c0_X

5   -258.058988
0   -290.831826
4   -234.212702
1   -207.776739
9   -182.436783
2   -194.994317
3   -281.485180
6   -103.680542
7   -209.260880
8   -152.192937
Name: 0, dtype: float64

In [32]:
def predict(X,P_class,E):
    bz1 = np.subtract(X,1)
    bz2 = np.abs(bz1)
    bz3 = np.subtract(bz2,E)
    bz4 = np.abs(bz3)
    log_p_fi_cj = np.log(bz4) 
    log_p_cj    = np.log(P_class.values)
    P_cj_X      = np.add(log_p_cj, np.sum(log_p_fi_cj,axis=1))
    return np.argmax(P_cj_X)

In [33]:
max_class = predict(sample_x,P_class,E)

In [34]:
max_class

7

In [35]:
bz1 = np.subtract(sample_x,1)
bz2 = np.abs(bz1)
bz3 = np.subtract(bz2,E)
bz4 = np.abs(bz3)
log_p_fi_cj = np.log(bz4) 
log_p_cj    = np.log(P_class.values)
P_cj_X      = np.add(log_p_cj, np.sum(log_p_fi_cj,axis=1))

In [36]:
P_cj_X

array([-258.0589881 , -290.8318261 , -234.21270244, -207.77673933,
       -182.43678332, -194.99431735, -281.4851801 , -103.680542  ,
       -209.26087979, -152.19293702])

In [40]:
df_test.shape[0]

10000

In [26]:
pre_list = []
for i in range(df_test.shape[0]):
    sample_x = df_test.iloc[i,1:].values.reshape(1,-1)
    pre_list.append(predict(sample_x,P_class,E))

In [27]:
y = df_test.iloc[:,0]
acc = len(y[y==pd.Series(pre_list)])/len(y)
acc

0.8428